In [113]:
#neural network from scratch

from random import randrange
from random import random
from csv import reader
from math import exp

In [117]:
# Initialising Network

def initialize_network(n_inputs, n_outputs, n_hidden):
    network = [[i for i in range(n_inputs)]]
    for i in range(len(n_hidden)):
        hidden=[{'weights' : [random() for j in range(len(network[i])+1)]} for k in range(n_hidden[i])]
        network.append(hidden)
    output = [{'weights' : [random() for j in range(len(network[-1])+1)]} for k in range(n_outputs)]
    network.append(output)
    network.pop(0)
    return network

def print_network(network):
    i=0
    for layer in network:
        print(i)
        i+=1
        for neuron in layer:
            print(neuron)

In [118]:
# Forward Propagate
#1. Neuron Activation
#2. Neuron Transfer
#3. Forward Propagate

def forward_propagate(network,input_row):
    input=input_row
    for i in range(len(network)):
        new_input=[]
        for neuron in network[i]:
            neuron['output'] = neuron_activation(input, neuron['weights'])
            new_input.append(neuron['output'])
        input=new_input
    return input

def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

def neuron_activation(input, weights):
    sum=weights[-1]
    for i in range(len(weights)-1):
        sum+=weights[i]*input[i]
    return transfer(sum)


In [119]:
# Back Propagation
#1. Transfer derivative
#2. Error Backpropagation

def transfer_derivative(output):
    #assuming transfer function is sigmoid
    return output * (1.0 - output)


def back_propagate(network, expected):

    for i in reversed(range(len(network))): #layer of network
        if i == len(network)-1 :
            for j in range (len(network[i])):  #neuron of layer
                neuron= network[i][j]
                neuron['error']=neuron['output']-expected[j]
        else:
            for j in range(len(network[i])): #neuron of layer
                neuron= network[i][j]
                error = 0
                for next_neuron in network[i+1]: #neuron of next layer
                    error+= next_neuron['delta'] * next_neuron['weights'][j]
                neuron['error'] = error

        for neuron in network[i]:
            neuron['delta'] = neuron['error'] * transfer_derivative(neuron['output'])


In [128]:
# Training network
#1. Update weights
#2. Train Network

def update_weights(network, inputs, learning_rate):
    previous = inputs[:-1]
    for i in range(len(network)):
        if i!= 0:
            previous = [neuron['output'] for neuron in network[i-1]]
        for neuron in network[i]:
            for j in range(len(previous)):
                neuron['weights'][j] -= learning_rate * neuron['delta'] * previous[j]
            neuron['weights'][-1] -= learning_rate* neuron['delta']
        previous= network[i]


def train_network(network, train, n_epochs, l_rate):
    for epoch in range(n_epochs):
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(len(network[-1]))]
            expected [row[-1]] = 1
            back_propagate(network, expected)
            update_weights(network, row, l_rate)



In [140]:
dataset = [[2.7810836,2.550537003,0],
[1.465489372,2.362125076,0], [3.396561688,4.400293529,0], [1.38807019,1.850220317,0], [3.06407232,3.005305973,0], [7.627531214,2.759262235,1], [5.332441248,2.088626775,1], [6.922596716,1.77106367,1], [8.675418651,-0.242068655,1], [7.673756466,3.508563011,1]]

n_inputs = 2
n_outputs = 2
network = initialize_network(n_inputs, n_outputs, [2, 2])
train_network(network, dataset,20, 0.5)


In [123]:
def predict(network, row):
    outputs = forward_propagate(network, row[:-1])
    return outputs.index(max(outputs))

In [142]:

n_inputs = 2
n_outputs = 2
network = initialize_network(n_inputs, n_outputs, [2] )
print_network(network)

train_network(network, dataset, 20, 0.5)
print_network(network)

for row in dataset:
    print('Expected=%d, Got=%d' % (row[-1], predict(network, row)))

0
{'weights': [0.1991942157934945, 0.1869388265371963, 0.005216829762140995]}
{'weights': [0.32091552872440143, 0.07125919973086914, 0.6128149523130053]}
1
{'weights': [0.8455446013622306, 0.8626939858504831, 0.09876073780394401]}
{'weights': [0.6475550652325738, 0.4777374516550541, 0.5320823703125206]}
0
{'weights': [-1.092630519281423, 1.349879979205579, 0.36584177848927596], 'output': 0.04608946986300928, 'error': 0.15917038490925814, 'delta': 0.0069979626821217444}
{'weights': [-1.09958116612838, 1.1590468173138118, 0.9216782687239142], 'output': 0.03740813972143541, 'error': 0.15533264203470346, 'delta': 0.005593337505410051}
1
{'weights': [1.8687934929724948, 1.5598310421401733, -1.317630999130397], 'output': 0.24028082824401997, 'error': 0.24028082824401997, 'delta': 0.043862292496476295}
{'weights': [-1.2840376424395818, -1.4462727606490553, 1.034147958166741], 'output': 0.709044317929257, 'error': -0.29095568207074296, 'delta': -0.060024294874399704}
Expected=0, Got=0
Expected

In [139]:
# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup

# Find the min and max values for each column
def dataset_minmax(dataset):
	minmax = list()
	stats = [[min(column), max(column)] for column in zip(*dataset)]
	return stats

# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)-1):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores


def back_propagation(train, test, l_rate, n_epoch, n_hidden):
	n_inputs = len(train[0]) - 1
	n_outputs = len(set([row[-1] for row in train]))
	network = initialize_network(n_inputs, n_outputs , n_hidden)
	train_network(network, train,n_epoch, l_rate )
	predictions = list()
	for row in test:
		prediction = predict(network, row)
		predictions.append(prediction)
	return(predictions)


filename = 'wheat-seeds.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0])-1):
	str_column_to_float(dataset, i)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
# normalize input variables
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)
# evaluate algorithm
n_folds = 5
l_rate = 0.5
n_epoch = 1000
n_hidden = [5, 4, 2]
scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [97.61904761904762, 95.23809523809523, 95.23809523809523, 90.47619047619048, 90.47619047619048]
Mean Accuracy: 93.810%
